In [22]:
import modules.singleton_reduction as singleton
import modules.isolate_reduction as isolate
from tqdm import tqdm
import importlib
importlib.reload(singleton)
importlib.reload(isolate)
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import io
import copy


In [23]:
# Load the full graph
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

In [24]:
importlib.reload(singleton)
importlib.reload(isolate)

G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
# isolatedNodes = list(nx.isolates(G))
G, isolatedNum = isolate.isolate_reduction(G)

Node count before singleton reduction:  58742
Nodes before:  52124
Nodes after:  19257


In [25]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
init_infected = [9]
max_steps = 100
G_temp = copy.deepcopy(G)

# Uncomment below block if you want to only view nodes that are accessible from init_infected
G_temp = G_temp.subgraph(sir.get_accessible_sus_nodes(G_temp, init_infected)).copy()

In [ ]:
# Uncomment below block if you want to only view nodes that are accessible from init_infected
# print("Nodes to calculate: ", len(G_temp))
# pos = nx.spring_layout(G_temp, seed=8020, gravity=0.75)

In [26]:
importlib.reload(sir)

_, _, _, _, infotext_total, constants_total = sir.sir_model(G_temp, G_full_sr, init_infected=init_infected, infection_rate=0.2, recovery_rate=0.1, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0.0001,0.01), network_type='full')

[DEBUG2] False
Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  1
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG

In [27]:
infotext_total

[{'ssw': 950,
  'sso': 5668,
  'it': 1,
  'st': 4990,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 5086, 'i': 220968},
  'afterloss': {'q': 5086, 'i': 91990},
  'i_prevented': 0,
  'ss': 6618,
  'is': 0,
  'rs': 0},
 {'ssw': 788,
  'sso': 5668,
  'it': 604,
  'st': 4387,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 5086, 'i': 220968},
  'afterloss': {'q': 5086, 'i': 91990},
  'i_prevented': 1888,
  'ss': 6456,
  'is': 162,
  'rs': 0},
 {'ssw': 661,
  'sso': 5668,
  'it': 1150,
  'st': 3841,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 5086, 'i': 220968},
  'afterloss': {'q': 5086, 'i': 91990},
  'i_prevented': 3969,
  'ss': 6329,
  'is': 289,
  'rs': 0},
 {'ssw': 554,
  'sso': 5668,
  'it': 1654,
  'st': 3337,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 5086, 'i': 220968},
  'afterloss': {'q': 5086, 'i': 91990},
  'i_prevented': 6465,
  'ss': 6222,
  'is': 396,
  'rs': 0},
 {'ssw': 471,
  'sso': 5668,
  'it': 2139,
  'st': 2852,
  'rt': 0,
  'qt': 0,
  'loss': {'q': 5086, 'i': 220968},
  'afterloss': {'q': 5086,

In [28]:
constants_total

{'virusFoundStep': 16, 'tn': 58742, 'xt': 53751}

In [29]:
# Graphical Analysis of a single run
plt.close()

# create data
x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

loss_total = [i['loss']['q']+i['loss']['i'] for i in infotext_total]
max_loss_total = max(loss_total)
loss_total = [i/max_loss_total*(constants_total['tn']-constants_total['xt']) for i in loss_total]


# plot lines
plt.plot(x, st_total, label = "Total susceptible", alpha=0.5, color='g')
plt.plot(x, it_total, label = "Total infected", alpha=0.5, color='r')
plt.plot(x, rt_total, label = "Total recovered", alpha=0.5, color='b')
plt.plot(x, qt_total, label = "Total quarantined", alpha=0.5, color='0.7')
plt.plot(x, loss_total, label = "Loss (scaled)", alpha=0.5, color='k')
plt.vlines(constants_total['virusFoundStep'], 0, constants_total['tn']-constants_total['xt'], color='k', linestyle='dotted', lw=1)
plt.legend()
plt.savefig(f"results/test.png", format="PNG", bbox_inches='tight')

In [30]:
# Loss analysis of a single run
plt.close()
loss = {'quarantine': 1, 'infected': 1}

x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

plt.stackplot(x, (
                    [i*loss['infected'] for i in it_total],
                    [i*loss['quarantine'] for i in qt_total]
                ),
             labels=['Loss from infected', 'Loss from quarantines'],
             colors=['#ff0000','#444444'],
              alpha=0.8)
plt.legend()
plt.savefig(f"results/test2.png", format="PNG", bbox_inches='tight')

In [ ]:
importlib.reload(sir)
import os
from multiprocessing import Pool
from functools import partial

# Loss analysis of multiple runs
run_count = 1000

init_infected = [9]
# init_infected = 
max_steps = 1000
infection_rate = 0.2
recovery_rate = 0.1
noticeability_rates = (0.01,0.1)
quarantine_length = 5
contact_tracing = 3
loss_matrix = ((1000, 0.5),(0.2, 0.1))

loss_results = None

tempList = []
for i in range(run_count):
    tempList.append(G)

worker = partial(
    sir.simulate_threaded,
    G=G, 
    G_full_sr=G_full_sr, 
    init_infected=init_infected, 
    infection_rate=infection_rate, 
    recovery_rate=recovery_rate, 
    max_steps=max_steps, 
    noticeability_rates=noticeability_rates, 
    quarantine_length=quarantine_length,
    contact_tracing=contact_tracing,
    loss_matrix=loss_matrix,
    tempList=tempList,
)

with Pool(16) as p:
    results = p.map(worker, range(run_count))

loss_results = [x['loss'] for x in results]
afterloss_results = [x['afterloss'] for x in results]
recovered_results = [x['rt'] for x in results]

total_loss_results = [x['q'] + x['i'] for x in loss_results] 
total_afterloss_results = [x['q'] + x['i'] for x in afterloss_results] 

print(f"Total Loss: {np.mean(total_loss_results)} -- Standard Dev.: {np.std(total_loss_results)}")
print(f"Infected Loss: {np.mean([x['i'] for x in loss_results])} -- Infected After-Loss: {np.mean([x['q'] for x in loss_results])}")
print(f"Total After-loess: {np.mean(total_afterloss_results)} -- Standard Dev.: {np.std(total_afterloss_results)}")
print(f"Infected After-Loss: {np.mean([x['i'] for x in afterloss_results])} -- Quarantined After-Loss: {np.mean([x['q'] for x in afterloss_results])}")
print(f"Average total infected nodes: {np.mean(recovered_results)}")

Run:  1
Run:  2
Run:  Run: 3
 16
Run:  17
Run:  4
Run:  18
Run:  5
Run:  32
Run:  19
Run:  6
Run:  33
Run:  20
Run:  7
Run:  34
Run:  21
Run:  48
Run:  8
Run:  35
Run:  22
Run:  49
Run:  36
Run:  9
Run:  23
Run:  10
Run:  24
Run:  51
Run:  64
Run:  37
Run:  52Run: 
 11
Run:  25
Run:  65
Run:  38
Run:  26
Run:  66
Run:  12
Run:  39
Run:  27
Run:  53
Run:  67
Run: Run:  80 
13
Run:  40
Run:  28
Run:  81
Run:  68
Run:  54
Run:  14
Run:  41
Run:  29
Run:  55
Run:  15
Run:  82
Run:  42
Run:  69
Run:  30
Run:  56
Run:  43
Run:  70
Run:  83
Run:  96
Run:  31
Run:  57
Run:  71
Run:  44
Run:  97
Run:  84
Run:  58
Run:  72
Run:  98
Run:  45
Run:  85
Run:  59
Run:  73
Run:  112
Run:  86
Run:  46
Run:  99
Run:  74
Run:  60
Run:  47
Run:  87
Run:  113
Run:  75
Run:  114
Run:  61
Run:  101
Run:  76
Run:  62
Run:  88Run: 
 115
Run:  128
Run:  102
Run:  77
Run:  63
Run:  116
Run:  89
Run:  129
Run:  78
Run:  117
Run:  90
Run:  103
Run:  79
Run:  130
Run:  144
Run:  118
Run:  91
Run:  145
Run:  104
Run

Run:  1
Run:  2
Run:  16
Run:  3
Run:  17
Run:  4
Run:  18
Run:  5
Run:  32
Run:  19
Run:  6
Run:  33
Run:  7
Run:  20
Run:  34
Run:  8
Run:  21
Run:  48
Run:  35
Run:  22
Run:  9
Run:  49
Run:  36
Run:  23
Run:  10
Run:  37
Run:  24
Run:  11
Run:  64
Run:  38
Run:  51
Run: Run:  65 
12
Run:  39
Run:  25
Run:  52
Run:  66
Run:  40
Run:  26
Run:  13
Run:  53
Run:  41
Run:  67
Run:  27
Run:  80
Run:  14
Run:  68
Run: Run:   5442

Run:  55
Run:  15
Run:  81
Run:  69
Run:  28
Run:  43
Run:  70
Run:  56
Run:  82
Run:  96
Run:  71
Run:  29
Run:  57
Run:  97
Run:  44
Run:  83
Run:  72
Run:  98
Run:  58
Run:  45
Run:  84
Run:  30
Run:  112
Run:  59
Run:  73
Run: Run:   9985

Run:  46
Run:  31
Run:  113
Run:  86
Run:  47
Run:  74
Run:  87
Run:  114
Run:  60
Run:  101
Run:  75
Run: Run:   12888

Run:  115
Run:  102
Run:  76
Run:  129
Run: Run:   61103Run: 

 89
Run:  116
Run:  77
Run:  130
Run:  62
Run:  Run: 117
 90
Run:  78
Run:  104
Run:  118
Run:  131
Run:  63
Run:  144
Run:  91
Run:  79
Run

Run:  1
Run:  2
Run:  16
Run:  3
Run:  17
Run:  4
Run:  18
Run:  5
Run:  32
Run:  19
Run:  6
Run:  33
Run:  20
Run:  7
Run:  34
Run:  8
Run:  21
Run:  48
Run:  35
Run:  22
Run:  9
Run:  49
Run:  23
Run:  36
Run:  10
Run:  24
Run:  37
Run:  11
Run:  64
Run:  38
Run:  25
Run:  12
Run:  39
Run: Run: Run:    512665


Run:  40
Run:  27
Run:  66
Run:  13
Run:  80
Run:  41
Run:  14
Run:  67
Run:  52
Run:  28
Run:  42
Run:  81
Run:  29
Run:  15
Run:  53
Run:  43
Run:  68
Run:  82
Run:  30
Run:  54
Run:  44
Run:  83
Run:  96
Run:  31
Run:  69
Run:  55
Run: 
 45
Run:  84
Run:  70
Run:  56
Run:  97
Run:  46
Run:  85
Run:  47
Run:  98
Run:  57
Run:  71
Run:  86
Run:  112
Run:  58Run:  99
Run:  72
Run:  87
Run:  113
Run:  59
Run:  88
Run:  73
Run:  114
Run:  60
Run:  89
Run:  101
Run: Run:  74 
115
Run:  61
Run:  128
Run:  90
Run:  102
Run:  75
Run:  62
Run:  116
Run:  91
Run:  63
Run:  76
Run:  129
Run:  103
Run:  92
Run:  144
Run:  117
Run:  77
Run:  130
Run:  93
Run:  118
Run:  104
Run:  145
Run

In [193]:
plt.cla()
plt.hist(total_loss_results, bins=200)
plt.savefig("results/test3.png")

# loss_matrix = ((1, 0),(0.2, 0.1)) w/ 100 runs
## max_steps = 100
quarantine_length 5: 15342.928
quarantine_length 5, ct level 2: 14599.878000000002
quarantine_length 10: 15261.373
quarantine_length 5, ct level 1, nt_rate * 10 noticed: 14576.289000000004
quarantine_length inf, ct level 1: 20178.598

## max_steps = 1000 (*)
quarantine_length 5, ct level 1: 14514.140000000009
quarantine_length 10, ct level 1: 15413.901000000007
quarantine_length 5, ct level 2: 14708.250000000007
quarantine_length 5, ct level 3: 15418.860000000004
quarantine_length 2, ct level 1: 15211.508000000003
no quarantines: 14869.428000000004

# loss_matrix = ((1, 0),(0.2, 0.1)) w/ 1000 runs
## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.2 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01)
no quarantining: ~14470 loss and ~3637 std
quarantine_length 5, ct level 0: ~14492 loss and ~3610 std
quarantine_length 10, ct level 0: ~14804 loss and ~3727 std
quarantine_length 5, ct level 1: ~14948 loss and ~3747 std
quarantine_length 10, ct level 1: ~15314 loss and ~3720 std
quarantine_length 5, ct level 2: ~14725 loss and ~3844 std
quarantine_length 10, ct level 2: ~15281 loss and ~4066 std
quarantine_length 5, ct level 3: ~14989 loss and ~3863 std
quarantine_length 10, ct level 3: ~15204 loss and ~3892 std

## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.1 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01) 
no quarantining: ~12231 loss and ~3892 std
quarantine_length 5, ct level 0: ~12197 loss and ~3997 std
quarantine_length 10, ct level 0: ~12284 loss and ~3977 std
quarantine_length 5, ct level 1: ~12591 loss and ~4007 std (q: 437 loss -- i: 11927)
quarantine_length 10, ct level 1: ~12502 loss and ~3913 std (q: 706 loss -- i: 11796)
quarantine_length 5, ct level 2: ~12287 loss and ~3809 std (q: 411 loss -- i: 11876)
quarantine_length 10, ct level 2: ~12673 loss and ~4074 std (q: 725 loss -- i: 11948)

# loss_matrix = ((1000, 0.5),(0.2, 0.1)) w/ 1000 runs
## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.2 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01)
no quarantining: ~3976276 loss and ~524603 std
quarantine_length 5, ct level 0: ~3942146 loss and ~547844 std (q: 350 -- i: 3941796)
quarantine_length 10, ct level 0: ~3966693 loss and ~531257 std (q: 490 -- i: 3966203)
quarantine_length 5, ct level 1: ~3920625 loss and ~538986 std (q: 986 -- i: 3919639)
quarantine_length 10, ct level 1: ~3918447 loss and ~530026 std (q: 1320 -- i: 3917127)
quarantine_length 5, ct level 2: ~3881514 loss and ~542880 std (q: 947 -- i: 3880568)
quarantine_length 10, ct level 2: ~3932767 loss and ~533736 std (q: 1298 -- i: 3931469)
quarantine_length 5, ct level 3: ~3973245 loss and ~502952 std (q: 938 -- i: 3972307)

quarantine_length 10, ct level 3: ~3942986 loss and ~531066 std (q: 949 -- i: 3942037)




7.865,665400001
